In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os
import time
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
import random
import shap
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from captum.attr import IntegratedGradients
import random

In [11]:
def plot_metrics(train, val, epochs, title):
    """
    Plot the training, validation, and test metrics.

    Parameters:
        - train_loss (list): Training metric
        - val_loss (list): Validation metric
        - test_loss (list): Test tmetric
        - epochs (int): The number of epochs.
    """
    sns.set_theme(style="whitegrid", palette="pastel")
    plt.figure(figsize=(8, 8))
    plt.plot(range(1, epochs + 1), train, label='Training', color=sns.color_palette("pastel")[0])
    plt.plot(range(1, epochs + 1), val, label='Validation', color=sns.color_palette("pastel")[1])
    #plt.plot(range(1, epochs + 1), test, label='Test', color=sns.color_palette("pastel")[2])
    plt.xlabel('Epochs')
    plt.ylabel(f"{title}")
    plt.title(f"{title}")
    plt.legend()
    plt.show()

In [12]:
class DataSampler:
    """
    DataSampler: Manages sampling and splitting of the USNW-NB15 dataset. Returns a training, validation, and test set.

    Initialisation:
        - train (None): The attributes that stores the train set.
        - val (None): The attributes that stores the validation set.
        - test (None): The attributes that stores the test set.
    """

    def __init__(self):
        self.train = None
        self.val = None
        self.test = None

    def sample_data(self, data, size, rs, target_label):
        """
        sample_data: Combines the USNW-NB15 1, 2, 3, 4 sets, does some basic cleaning, Use a random number generator to take slices for the validation
        and test set from the main data then splits into train, validation, and test set. Downsamples the data so that Normal labels match Generic in each set.
        Uses a mask to take only Normal values that do not precede a Non-Normal value.

        Parameters:
            - data (string): The dataset to use (currently only USNW-NB15 is supported).
            - type_of (string):  The type of sampling.
            - rs (int): The random seed for slicing operation.

        NOTE: Includes print outs to verify the process.
        """

        if data == 'full_data':
            feature_names = pd.read_csv('features2.csv' )

            #print('Set1')
            feature_names_list = feature_names['Name'].tolist()
            s1 = pd.read_csv('UNSW-NB15_1.csv', header=None)
            s1.columns = feature_names_list
            s1.loc[s1['attack_cat'].isnull(), 'attack_cat'] = 'Normal'
            #print(s1['attack_cat'].value_counts())
            #print('Set2')
            feature_names_list = feature_names['Name'].tolist()
            s2 = pd.read_csv('UNSW-NB15_2.csv', header=None)
            s2.columns = feature_names_list
            s2.loc[s2['attack_cat'].isnull(), 'attack_cat'] = 'Normal'
            #print(s2['attack_cat'].value_counts())
            #print('Set3')
            feature_names_list = feature_names['Name'].tolist()
            s3 = pd.read_csv('UNSW-NB15_3.csv', header=None)
            s3.columns = feature_names_list
            s3.loc[s3['attack_cat'].isnull(), 'attack_cat'] = 'Normal'
            #print(s3['attack_cat'].value_counts())
            #print('Set4')
            feature_names_list = feature_names['Name'].tolist()
            s4 = pd.read_csv('UNSW-NB15_4.csv', header=None)
            s4.columns = feature_names_list
            s4.loc[s4['attack_cat'].isnull(), 'attack_cat'] = 'Normal'
            #print(s4['attack_cat'].value_counts())
            data = [s1, s2, s3, s4]
            i = 0

            while i < len(data):
                df = data[i]
                #print(f"Set{i + 1}")
                normal = df[df['attack_cat'] == 'Normal'].shape[0]
                generic = df[df['attack_cat'] == 'Generic'].shape[0]
                difference = normal - generic
                mask = df['Label'].shift(-1) != 1
                rows = df[(df['attack_cat'] == 'Normal') & mask]
                downsampled = rows.sample(n=difference, random_state=rs)
                df2 = df.drop(downsampled.index)
                #print(df2['attack_cat'].value_counts())
                data[i] = df2
                i += 1
            # Clean Labels.
            full_data = pd.concat([data[0], data[1], data[2], data[3]]).reset_index(drop=True)
            full_data['attack_cat'] = full_data['attack_cat'].str.replace(r'\s+', '', regex=True)
            full_data['attack_cat'] = full_data['attack_cat'].str.replace('Backdoors', 'Backdoor')
            # Drop Sparse Data.
            # NOTE: These values could also be transformed by adding 1 and setting nulls to 0.
            full_data = full_data.drop(columns=['ct_ftp_cmd', 'ct_flw_http_mthd', 'is_ftp_login'])
            # Remove error values created by nulls.
            full_data['sport'] = full_data['sport'].apply(pd.to_numeric)
            full_data = full_data[~full_data['dsport'].astype(str).str.startswith('0x')]
            full_data['dsport'] = full_data['dsport'].apply(pd.to_numeric)
            full_data['attack_cat'] = (full_data['attack_cat'] == target_label).astype(int)
            slice_size = int(size * len(full_data))
            val_start = random.randrange(0, len(full_data) - 2 * slice_size)
            val_end = val_start + slice_size
            val_data = full_data.iloc[val_start:val_end]
            df = full_data.drop(val_data.index)
            test_start = random.randrange(0, len(df) - slice_size)
            test_end = test_start + slice_size
            test_data = df.iloc[test_start:test_end]
            train_data = df.drop(test_data.index)

            #print('Train')
            #print(len(train_data))
            #print('Val')
            #print(len(val_data))
            #print('Test')
            #print(len(test_data))
            
            return train_data, val_data, test_data
    

In [13]:
class DataManager:
    """
    DataManager: Help manage the pre-processing of the USNW-NB15 dataset for an LSTM.

    Initialisation:
        - train (pd.Dataframe): The training data.
        - val (pd.Dataframe): The validation data.
        - test (pd.Dataframe): The teting data.
        - type_of (string): Specify if multiclass or other (currently only multi-class is supported).
    """
    def __init__(self, train, val, test, type_of):
        self.train = train
        self.val = val
        self.test = test
        if type_of == 'multi':
            for i in [self.train, self.val, self.test]:
                i.drop(columns=['Label'], inplace=True)    
            self.label = 'attack_cat'
        if type_of == 'binary':
            for i in [self.train, self.val, self.test]:
                i.drop(columns=['attack_cat'], inplace=True)    
            self.label = 'Label'
        
    def label_encode(self):
        """
        label_encode: label encodes the categorical columns. Preset to 'proto', 'state', and 'service' and the label.

        NOTE: This can have conflicts depending on what seed is used in the sampler.

        """

        data = [self.train, self.val, self.test]
        columns = ['srcip', 'dstip', 'proto', 'state', 'service', self.label]
        encoder = LabelEncoder()
        for col in columns:
            for df in data:
                df[col] = encoder.fit_transform(df[col])
        self.train, self.val, self.test = data
    
    def onehot_encode(self):
        """
        onehot_encode: One-hot encodes the categorical columns. Preset to 'proto', 'state', and 'service'. Label encodes the target.

        NOTE: This can have conflicts depending on what seed is used in the sampler.
        """
        columns = ['srcip', 'dstip', 'proto', 'state', 'service']

        encoder2 = LabelEncoder()
        encoder = OneHotEncoder(sparse_output=False, dtype='float32')

        for col in columns:
            encoded_train = encoder.fit_transform(self.train[[col]])
            encoded_val = encoder.transform(self.val[[col]])
            encoded_test = encoder.transform(self.test[[col]])
            encoded_train_df = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out([col]), index=self.train.index)
            encoded_val_df = pd.DataFrame(encoded_val, columns=encoder.get_feature_names_out([col]), index=self.val.index)
            encoded_test_df = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out([col]), index=self.test.index)
            self.train = pd.concat([self.train.drop(columns=[col]), encoded_train_df], axis=1)
            self.val = pd.concat([self.val.drop(columns=[col]), encoded_val_df], axis=1)
            self.test = pd.concat([self.test.drop(columns=[col]), encoded_test_df], axis=1)

        encoder = LabelEncoder()
        for df in [self.train, self.val, self.test]:
            df[self.label] = encoder.fit_transform(df[self.label])


    def standardise(self, type_of):
        """
        standardise: Standardises the features of the train and test datasets.

        Parameters:
            - type_of (string): Choose the standardisation ('standard' or 'minmax').
        """
        train_features = self.train.drop(columns=[self.label])
        val_features = self.val.drop(columns=[self.label])
        test_features = self.test.drop(columns=[self.label])
        train_label = self.train[self.label]
        val_label = self.val[self.label]
        test_label = self.test[self.label]
        if type_of == 'minmax':
            scaler = MinMaxScaler()
        elif type_of == 'standard':
            scaler = StandardScaler()
        scaled_train = scaler.fit_transform(train_features)
        scaled_val = scaler.transform(val_features)
        scaled_test = scaler.transform(test_features)
        self.train = pd.DataFrame(scaled_train, columns=train_features.columns, index=self.train.index)
        self.train[self.label] = train_label
        self.val = pd.DataFrame(scaled_val, columns=val_features.columns, index=self.val.index)
        self.val[self.label] = val_label
        self.test = pd.DataFrame(scaled_test, columns=test_features.columns, index=self.test.index)
        self.test[self.label] = test_label

    def get_sequence(self):
        """
        get_sequence: Converts the Dtaframe to a sequence for each row and the corresponding label.
        
        Parameters:
            - batch_size (int): The batch size to use in DataLoader.

        Returns:
            - train_loader (DataLoader): Dataloader for the training data.
            - val_loader (DataLoader): Dataloader for the validation data.
            - test_loader (DataLoader): Dataloader for the test data.
        """
        self.test = self.test.sample(n=1000, random_state=42)
        train_seq = torch.tensor(self.train.drop(columns=[self.label]).values, dtype=torch.float32)
        val_seq = torch.tensor(self.val.drop(columns=[self.label]).values, dtype=torch.float32)
        test_seq = torch.tensor(self.test.drop(columns=[self.label]).values, dtype=torch.float32)
        train_label = torch.tensor(self.train[self.label].values, dtype=torch.float32)
        val_label = torch.tensor(self.val[self.label].values, dtype=torch.float32)
        test_label = torch.tensor(self.test[self.label].values, dtype=torch.float32)

        # Add dimension for LSTM.
        train_seq = train_seq.unsqueeze(1)
        val_seq = val_seq.unsqueeze(1)
        test_seq = test_seq.unsqueeze(1)

        train_dataset = TensorDataset(train_seq, train_label)
        val_dataset = TensorDataset(val_seq, val_label)
        test_dataset = TensorDataset(test_seq, test_label)
        
        return train_dataset, val_dataset, test_dataset
    
    def get_weights(self, type_of):
        """
        get_weights: Calculate the weights of the labels for the given dataset. Used in Cross Entropy Loss criterion.
        
        Parameters:
            - type_of (string): The dataset to use (train, val, test).

        Returns:
            - class_weights (tensor): The class weights for the label.
        """
        if type_of == 'train':
            data = self.train
        elif type_of == 'val':
            data = self.val
        elif type_of == 'test':
            data = self.test
        labels = data[self.label].values
        class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
        return torch.tensor(class_weights, dtype=torch.float)

In [15]:
class LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, device, save_dir):
        super(LSTM_Model, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.device = device
        self.save_dir = save_dir
        self.to(self.device)
        self.train_loss = []
        self.val_loss = []
        self.train_accuracy = []
        self.val_accuracy = []
        self.train_f1 = []
        self.val_f1 = []
        self.epoch_time = []
        self.test_accuracy = None
        self.test_loss = None
        self.test_f1 = None
        self.test_predicted = []
        self.test_actual = []

    def forward(self, x):
        """
        forward: The forward pass for the LSTM. Calculates the label for the same row.
        This is a magic method called when the model is called (model(input)).

        Parameters:
            - x (tensor): The input sequence for the LSTM.

        Returns:
            - output (tensor): Returns the output after the LSTM layer.
        """
        lstm_out, _ = self.lstm(x)
        final_hidden_state = lstm_out[:, -1, :]
        output = self.fc(final_hidden_state)
        return output

    def calculate_accuracy_f1(self, predictions, labels):
        """
        calculate_accuracy_f1: A helper function to help modularize the code. Calculates accuracy and
        f1 macro (average).

        Parameters:
            - predictions (tensor): The predictions given by BCEWithLogitsLoss()
            - labels (tensor): The actual labels of the test data.

        Returns:
            - accuracy (float): The accuracy score.
            - f1 (float): The macro f1 score.
        """
        preds = torch.sigmoid(predictions)
        preds = (preds > 0.5).float()
        accuracy = accuracy_score(labels.cpu(), preds.cpu())
        f1 = f1_score(labels.cpu(), preds.cpu(), average='weighted')
        return accuracy, f1

    def run(self, train_data, val_data, test_data, criterion, optimizer, epochs, save_factor):
        """
        run: Run the model on the train, val data for each epoch. Evaluate on the test data at the end
        of all epochs. Returns an explainer object based on the gradient of all datapoints in the test set and model.

        Parameters:
            - train_data (TensorDataset): The tensor holding the train features and labels.
            - val_data (TensorDataset): The tensor holding the validation features and labels.
            - test_data (TensorDataset): The tensor holding the test features and labels.
            - criterion (nn.BCEWithLogitsLoss): The probability calculation for logits.
            - optimizer (optim.Adam): The automatic regularization optimizer.
            - epochs (int): The number of epochs to train.
            - save_factor (int): The factor at which to save the model during training.

        Returns:
            - explainer (shap.GradientExplainer): The calculated explainer for all data on the
            test set.
        """
        # Split the tensors.
        train_seq, train_label = train_data.tensors
        val_seq, val_label = val_data.tensors
        test_seq, test_label = test_data.tensors
        train_seq, train_label = train_seq.to(self.device), train_label.to(self.device)
        val_seq, val_label = val_seq.to(self.device), val_label.to(self.device)
        test_seq, test_label = test_seq.to(self.device), test_label.to(self.device)

        for epoch in range(epochs):
            start_time = time.time()
            self.train()
            optimizer.zero_grad()
            # Forward pass.
            outputs = self(train_seq)
            loss = criterion(outputs.squeeze(), train_label)
            loss.backward()
            optimizer.step()
            train_accuracy, train_f1 = self.calculate_accuracy_f1(outputs, train_label)

            self.eval()
            with torch.no_grad():
                val_outputs = self(val_seq)
                val_loss = criterion(val_outputs.squeeze(), val_label)
                val_accuracy, val_f1 = self.calculate_accuracy_f1(val_outputs, val_label)
            self.train_loss.append(loss.item())
            self.val_loss.append(val_loss.item())
            self.train_accuracy.append(train_accuracy)
            self.val_accuracy.append(val_accuracy)
            self.train_f1.append(train_f1)
            self.val_f1.append(val_f1)
            epoch_time = time.time() - start_time
            self.epoch_time.append(epoch_time)
            print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.4f}, Train F1: {train_f1:.4f}")
            print(f"Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}")
            if (epoch + 1) % save_factor == 0:
                self.save_model(epoch + 1)
        self.eval()
        with torch.no_grad():
            test_outputs = self(test_seq)
            test_loss = criterion(test_outputs.squeeze(), test_label)
            test_accuracy, test_f1 = self.calculate_accuracy_f1(test_outputs, test_label)
            self.test_accuracy = test_accuracy
            self.test_loss = test_loss.item()
            self.test_f1 = test_f1

            self.test_pred = (torch.sigmoid(test_outputs) > 0.5).cpu().numpy()
            self.test_actual = test_label.cpu().numpy()
            print(f"Test Loss: {self.test_loss:.4f}, Test Accuracy: {self.test_accuracy:.4f}, Test F1: {self.test_f1:.4f}")

        # NOTE: We cannot use no grad with SHAP explainers due to conflicts.
        explainer = shap.GradientExplainer(self, test_seq)
        return explainer

    def save_model(self, epoch):
        """
        save_model: Save the model state_dict for the given factor. Approx 1 megabyte each.

        Parameters:
            - The current epoch that is being saved.
        """
        checkpoint_path = os.path.join(self.save_dir, f"lstm_epoch_{epoch}.pt")
        torch.save({
            'epoch': epoch,
            'model_state_dict': self.state_dict(),
        }, checkpoint_path)

    def load_model(self, epoch):
        """
        load_model: Load a state_dict of a model.

        Parameters:
            - epoch (int): The epoch to load.
        """
        checkpoint_path = os.path.join(self.save_dir, f"lstm_epoch_{epoch}.pt")
        checkpoint = torch.load(checkpoint_path)
        self.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
class BinaryEnsembler:
    def __init__(self, labels):
        self.labels = labels
        self.data_ensemble = []
        self.model_ensemble = []
        self.explainers_ensemble = []
        self.tree_ensemble = []
        for label in self.labels:
            df = DataSampler()
            train_data, val_data, test_data = df.sample_data('full_data', 0.2, 42, label)
            dm = DataManager(train_data, val_data, test_data, 'multi')
            dm.label_encode()
            dm.standardise('minmax')
            train_data, val_data, test_data = dm.get_sequence()
            device = torch.device("cpu")
            model = LSTM_Model(44, 128, 1, device, 'model_ensemble')
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=0.01)
            explainer = model.run(train_data, val_data, test_data, criterion, optimizer, 10, 50)
            self.model_ensemble.append(model)
            self.explainers_ensemble.append(explainer)
            tr_data, labels = train_data.tensors
            tr_data = tr_data.squeeze()
            tr_data = pd.DataFrame(tr_data)
            tr_data['labels'] = pd.DataFrame(labels)
            te_data, labels = test_data.tensors
            te_data = te_data.squeeze()
            te_data = pd.DataFrame(te_data)
            te_data['labels'] = pd.DataFrame(labels)
            X_train = tr_data.drop(columns='labels')
            y_train = tr_data['labels']
            X_test = te_data.drop(columns='labels')
            y_test = te_data['labels']
            rf = RandomForestClassifier(n_estimators=100, random_state=42)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            print(f'Accuracy: {accuracy:.4f}')
            print("\nClassification Report:")
            print(classification_report(y_test, y_pred))
            self.tree_ensemble.append(rf)


In [19]:
be = BinaryEnsembler(['Dos', 'Fuzzers'])

C:\Users\adi_s\AppData\Local\Temp\ipykernel_17404\3378984335.py:35: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  s1 = pd.read_csv('UNSW-NB15_1.csv', header=None)
C:\Users\adi_s\AppData\Local\Temp\ipykernel_17404\3378984335.py:41: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  s2 = pd.read_csv('UNSW-NB15_2.csv', header=None)


Epoch [1/10], Train Loss: -0.0000, Train Accuracy: 0.0000, Train F1: 0.0000
Val Loss: -0.0000, Val Accuracy: 0.0000, Val F1: 0.0000
Epoch [2/10], Train Loss: -0.0000, Train Accuracy: 0.0000, Train F1: 0.0000
Val Loss: -0.0000, Val Accuracy: 0.0000, Val F1: 0.0000
Epoch [3/10], Train Loss: -0.0000, Train Accuracy: 0.0000, Train F1: 0.0000
Val Loss: -0.0000, Val Accuracy: 0.0000, Val F1: 0.0000
Epoch [4/10], Train Loss: -0.0000, Train Accuracy: 0.0000, Train F1: 0.0000
Val Loss: -0.0000, Val Accuracy: 0.0000, Val F1: 0.0000
Epoch [5/10], Train Loss: -0.0000, Train Accuracy: 0.0000, Train F1: 0.0000
Val Loss: -0.0000, Val Accuracy: 0.0000, Val F1: 0.0000
Epoch [6/10], Train Loss: -0.0000, Train Accuracy: 0.0000, Train F1: 0.0000
Val Loss: -0.0000, Val Accuracy: 0.0000, Val F1: 0.0000
Epoch [7/10], Train Loss: -0.0000, Train Accuracy: 0.0000, Train F1: 0.0000
Val Loss: -0.0000, Val Accuracy: 0.0000, Val F1: 0.0000
Epoch [8/10], Train Loss: -0.0000, Train Accuracy: 0.0000, Train F1: 0.0000


C:\Users\adi_s\AppData\Local\Temp\ipykernel_17404\3378984335.py:35: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  s1 = pd.read_csv('UNSW-NB15_1.csv', header=None)
C:\Users\adi_s\AppData\Local\Temp\ipykernel_17404\3378984335.py:41: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  s2 = pd.read_csv('UNSW-NB15_2.csv', header=None)


Epoch [1/10], Train Loss: 201484.8906, Train Accuracy: 0.0493, Train F1: 0.0046
Val Loss: 42903.0742, Val Accuracy: 0.4401, Val F1: 0.5748
Epoch [2/10], Train Loss: 200386.3750, Train Accuracy: 0.4572, Train F1: 0.5786
Val Loss: 42604.7734, Val Accuracy: 0.6842, Val F1: 0.7830
Epoch [3/10], Train Loss: 199169.8906, Train Accuracy: 0.6798, Train F1: 0.7701
Val Loss: 42227.5078, Val Accuracy: 0.7975, Val F1: 0.8603
Epoch [4/10], Train Loss: 197643.8125, Train Accuracy: 0.7479, Train F1: 0.8188
Val Loss: 41754.2500, Val Accuracy: 0.8255, Val F1: 0.8780
Epoch [5/10], Train Loss: 195754.3750, Train Accuracy: 0.7719, Train F1: 0.8352
Val Loss: 41186.5039, Val Accuracy: 0.8404, Val F1: 0.8873
Epoch [6/10], Train Loss: 193533.9531, Train Accuracy: 0.7842, Train F1: 0.8434
Val Loss: 40544.1406, Val Accuracy: 0.8502, Val F1: 0.8934
Epoch [7/10], Train Loss: 191101.2188, Train Accuracy: 0.7937, Train F1: 0.8497
Val Loss: 39869.2891, Val Accuracy: 0.8599, Val F1: 0.8994
Epoch [8/10], Train Loss: 1